In [1]:
#Import Modules
import os, sys, sqlite3
import pandas as pd


In [2]:
# creating file (path) name
dbfile = './DATA/BINA_DATA.db' 

# Test if the database file is available in the colab workspace
if os.path.exists(dbfile):
    # Create database (file) and Open a (SQL) connection 
    connection = sqlite3.connect(dbfile)
    # Create a data cursor to exchange information between Python and SQLite
    cursor = connection.cursor()
else:
    print("Angegebene Database wurde nicht gefunden")
    # Create database (file) and Open a (SQL) connection 
    connection = sqlite3.connect(dbfile)
    # Create a data cursor to exchange information between Python and SQLite
    cursor = connection.cursor()
    #sys.exit(0)


Angegebene Database wurde nicht gefunden


In [3]:
#Tabelle erstellen
sql = ["CREATE TABLE city (id INTEGER PRIMARY KEY, plz int,cityName string,bfsID int,lawCityName string)",
"CREATE TABLE smartmeter (id INTEGER PRIMARY KEY,timestamp timestamp,anzMeter int,valueKwh float, plz REFERENCES city(plz) ON UPDATE CASCADE)",
"CREATE TABLE solarPlants (id INTEGER PRIMARY KEY, xtfID int,  plz int,  totalPower float,  mainCategory string,  subCategory string, plantCategory string,  _x int,  _y int, FOREIGN KEY (plz) REFERENCES city(plz) ON UPDATE CASCADE, FOREIGN KEY (subCategory) REFERENCES subCategory(id) ON UPDATE CASCADE, FOREIGN KEY (mainCategory) REFERENCES mainCategory(id) ON UPDATE CASCADE, FOREIGN KEY (plantCategory) REFERENCES plantCategory(id) ON UPDATE CASCADE)",
"CREATE TABLE mainCategory (  id string PRIMARY KEY,  de string,  fr string,  it string,  en string)",
"CREATE TABLE plantCategory (  id string PRIMARY KEY,  de string,  fr string,  it string,  en string)",
"CREATE TABLE subCategory (  id string PRIMARY KEY,  de string,  fr string,  it string,  en string)",
"CREATE TABLE indicator (  id string PRIMARY KEY,  descr string)",
"CREATE TABLE unit (  id string,  mes string)",
"CREATE TABLE demoValue (  id INTEGER PRIMARY KEY,  bfsID int,  period string,  indicator string,  unit string,  value float, FOREIGN KEY (bfsID) REFERENCES plz(bfsID) ON UPDATE CASCADE, FOREIGN KEY (indicator) REFERENCES indicator (id) ON UPDATE CASCADE, FOREIGN KEY (unit) REFERENCES unit(id) ON UPDATE CASCADE)"]

for code in sql:
    cursor.execute(code)
    

In [4]:
#City Daten einlesen als Dataframe und in DB sichern
city = pd.read_csv("./DATA/city_directory/AMTOVZ_CSV_LV95.csv", delimiter=";")
city.head()
for index, row in city.iterrows():
    print(row["PLZ"])
    ort = '"' + row["Ortschaftsname"] + '"'
    lawCityName = '"' + row["Gemeindename"] + '"'
    sql = "INSERT INTO city VALUES(NULL,{},{},{},{})".format(row["PLZ"], ort, row["BFS-Nr"], lawCityName)
    cursor.execute(sql)

connection.commit()

8914
8914
8909
8910
8906
6340
8135
8915
8925
8908
8926
8926
8926
8934
8933
8932
8910
8912
8913
8911
8134
8143
8143
8907
8463
8415
8415
8414
8447
8458
8245
8246
8416
8247
8444
8451
8453
8461
8212
8248
8447
8460
8464
8466
8475
8462
8464
8478
8465
8465
8466
8467
8459
8180
8184
8303
8180
8192
8305
8600
8193
8197
8424
8427
8428
8180
8192
8192
8193
8182
8192
8181
8194
8302
8309
8426
8309
8425
8052
8152
8152
8152
8197
8427
8428
8304
8195
8196
8196
8197
8180
8185
8164
8113
8158
8107
8108
8114
8157
8115
8162
8164
8173
8174
8155
8172
8155
8155
8156
8166
8154
5462
8165
8107
8112
8158
8105
8105
8106
8052
8152
8153
8165
8165
8174
8175
8155
8162
8162
8187
8340
8344
8345
8498
8608
8633
8340
8608
8630
8632
8635
8636
8340
8494
8496
8497
8498
8614
8617
8624
8625
8626
8627
8627
8340
8342
8498
8630
8636
8607
8497
8636
8637
8345
8620
8623
8041
8134
8802
8803
8135
8942
8805
8833
8803
8136
8800
8703
8703
8704
8627
8634
8714
8127
8700
8618
8708
8706
8618
8634
8712
8713
8707
8126
8125
8702
8308
8320
8335
8310


In [5]:
#Solar Anlagen Daten einlesen als Dataframe und in DB sichern
plantCategory = pd.read_csv("./DATA/solar_powerplants/PlantCategoryCatalogue.csv", delimiter=",")
plantCategory.head()

for index, row in plantCategory.iterrows():
    catalogueId = '"' + row["Catalogue_id"] + '"'
    de = '"' + row["de"] + '"'
    fr = '"' + row["fr"] + '"'
    it = '"' + row["it"] + '"'
    en = '"' + row["en"] + '"'
    sql = "INSERT INTO plantCategory VALUES({},{},{},{},{})".format(catalogueId, de, fr, it, en)
    cursor.execute(sql)

connection.commit()

In [6]:
mainCategory = pd.read_csv("./DATA/solar_powerplants/MainCategoryCatalogue.csv", delimiter=",")
mainCategory.head()

for index, row in mainCategory.iterrows():
    catalogueId = '"' + row["Catalogue_id"] + '"'
    de = '"' + row["de"] + '"'
    fr = '"' + row["fr"] + '"'
    it = '"' + row["it"] + '"'
    en = '"' + row["en"] + '"'
    sql = "INSERT INTO mainCategory VALUES({},{},{},{},{})".format(catalogueId, de, fr, it, en)
    cursor.execute(sql)

connection.commit()

In [7]:
subCategory = pd.read_csv("./DATA/solar_powerplants/SubCategoryCatalogue.csv", delimiter=",")
subCategory.head()

for index, row in subCategory.iterrows():
    catalogueId = '"' + row["Catalogue_id"] + '"'
    de = '"' + row["de"] + '"'
    fr = '"' + row["fr"] + '"'
    it = '"' + row["it"] + '"'
    en = '"' + row["en"] + '"'
    sql = "INSERT INTO subCategory VALUES({},{},{},{},{})".format(catalogueId, de, fr, it, en)
    cursor.execute(sql)

connection.commit()

In [31]:
import numpy as np

electricityProductionPlant = pd.read_csv("./DATA/solar_powerplants/ElectricityProductionPlant.csv", delimiter=",")
electricityProductionPlant.head()
electricityProductionPlant.replace(np.nan, "NULL", inplace=True)
print(electricityProductionPlant.count())

for index, row in electricityProductionPlant.iterrows():
    mainCategoryId = '"' + row["MainCategory"] + '"'
    subCategoryId = '"' + row["SubCategory"] + '"'
    xCor = row["_x"]
    yCor = row["_y"]
    
    if isinstance(row["PlantCategory"], str):
        plantCategoryId = '"' + row["PlantCategory"] + '"'
    else:
        plantCategoryId = "NULL"
        
    sql = "INSERT INTO solarPlants VALUES(NULL,{},{},{},{},{},{},{},{})".format(row["xtf_id"], row["PostCode"], row["TotalPower"], mainCategoryId, subCategoryId, plantCategoryId, xCor, yCor)
    cursor.execute(sql)
    
connection.commit()

xtf_id                  203474
Address                 203474
PostCode                203474
Municipality            203474
Canton                  203474
BeginningOfOperation    203474
InitialPower            203474
TotalPower              203474
MainCategory            203474
SubCategory             203474
PlantCategory           203474
_x                      203474
_y                      203474
dtype: int64
